In [ ]:
import tensorflow as tf
import numpy as np



2.20.0


In [23]:
test_data = tf.keras.utils.image_dataset_from_directory(
    directory="../data/archive/test",  #? charge les images depuis ton dossier test
    labels='inferred',                 #? déduit les labels à partir des noms de sous-dossiers (angry, happy, etc.).
    label_mode='int',                  #? les labels seront des entiers (0, 1, 2...).
    image_size=(256, 256),             #? redimensionne toutes les images à 256×256.
    interpolation='nearest',
    batch_size=32,                     #? traite 32 images par batch.
    shuffle=True                       #? melange les images.
)

print("Classes :", test_data.class_names)


Found 7178 files belonging to 7 classes.
Classes : ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [19]:
train_data = tf.keras.utils.image_dataset_from_directory(
    directory="../data/archive/train",  
    labels='inferred',                
    label_mode='int',              
    image_size=(256, 256),           
    interpolation='nearest',
    batch_size=32,                    
    shuffle=True                       
)

print("Classes :", train_data.class_names)

Found 647 files belonging to 1 classes.
Classes : ['angry']


In [24]:
class_name = test_data.class_names
class_name

['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

In [28]:
len(test_data)


225

In [29]:
len(train_data)

21

In [35]:

#! Verifier size d'images
for images, labels in test_data.take(1):  #? prend juste un batch
    print("Batch shape :", images.shape)
    print("Label shape :", labels.shape)
    print("Première image :", images[0].shape)
    print("Premier label :", labels[0].numpy())


Batch shape : (32, 256, 256, 3)
Label shape : (32,)
Première image : (256, 256, 3)
Premier label : 2


2025-11-03 15:45:39.395575: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
